In [22]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler


import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Sequential (
            # 32-3+1=30*30*16
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3),
            nn.ReLU(inplace=True),
            # 30-3+1=28*28*16
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),
            nn.ReLU(inplace=True),
            # 28-3+1=26*26*64
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            # 13*13*64
            nn.MaxPool2d(2,2),
            nn.ReLU(inplace=True),
            # 13-2+1=12*12*128
            nn.Conv2d(in_channels=64,out_channels=128, kernel_size=2),
            nn.BatchNorm2d(128),
            # 6*6*128
            nn.MaxPool2d(2,2),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.05)
        )

        self.fc = nn.Sequential(
            nn.Linear(4608, 1024),
            nn.Dropout(0.05),
            nn.Linear(1024,128)
        )


    def forward(self, x):
        x = x.cuda()
        x = self.conv(x)
        x = x.view(x.shape[0],-1)
        x = self.fc(x)
        return x

PATH = './model.pth'

def main():
    batch_size, num_epoch = 8, 15
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    print(device)
    # load and transform dataset
    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                            shuffle=False, num_workers=2)

    classes = ('plane', 'car', 'bird', 'cat',
            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    net = CNN()
    net.to(device)
   
    criterion = nn.CrossEntropyLoss().to(device)
    # optimizer = optim.Adam(net.parameters(), lr=0.002)
    # # torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
    # scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

    optimizer = optim.SGD(net.parameters(), lr=0.0025, momentum=0.9)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

    for epoch in range(num_epoch):  # loop over the dataset multiple times
        for param_group in optimizer.param_groups:
            print("Epoch {}, learning rate: {}".format(epoch, param_group['lr']))
        scheduler.step(epoch)

        running_loss = 0.0
        
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 1000 == 999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    print('Finished Training')

    torch.save(net.state_dict(), PATH)


    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))

           
if __name__ == "__main__":
    main()



cuda
Files already downloaded and verified
Files already downloaded and verified
Epoch 0, learning rate: 0.0025
[1,  1000] loss: 1.023
[1,  2000] loss: 0.837
[1,  3000] loss: 0.742
[1,  4000] loss: 0.671
[1,  5000] loss: 0.628
[1,  6000] loss: 0.604
Epoch 1, learning rate: 0.0025
[2,  1000] loss: 0.560
[2,  2000] loss: 0.525
[2,  3000] loss: 0.523
[2,  4000] loss: 0.509
[2,  5000] loss: 0.486
[2,  6000] loss: 0.497
Epoch 2, learning rate: 0.0025
[3,  1000] loss: 0.447
[3,  2000] loss: 0.448
[3,  3000] loss: 0.442
[3,  4000] loss: 0.443
[3,  5000] loss: 0.439
[3,  6000] loss: 0.433
Epoch 3, learning rate: 0.0025
[4,  1000] loss: 0.382
[4,  2000] loss: 0.385
[4,  3000] loss: 0.385
[4,  4000] loss: 0.408
[4,  5000] loss: 0.395
[4,  6000] loss: 0.407
Epoch 4, learning rate: 0.0025
[5,  1000] loss: 0.348
[5,  2000] loss: 0.352
[5,  3000] loss: 0.350
[5,  4000] loss: 0.368
[5,  5000] loss: 0.359
[5,  6000] loss: 0.367
Epoch 5, learning rate: 0.0025
[6,  1000] loss: 0.311
[6,  2000] loss: 0.3